# Assignment 6

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [2]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [3]:
def exp_return(data, P):
    exp = sum(((row['Price in 1 Year']) / P - 1) * row['Probability'] for index, row in data.iterrows())
    return exp

Write the function ```volatility```, which calculates the expected return of the stock, given the current stock price ```P```.

In [4]:
def volatility(data, P):
    # Write your code here
    # volatility(data,40) should output 0.21301408404140795
    expected = exp_return(data, P)
    variation = 0
    for i in data.index:
        variation += (data["Probability"][i] * ((data["Price in 1 Year"][i] / P - 1) - expected) ** 2)
    volatility = variation ** (1/2) #sqrts variation
    return volatility

Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [5]:
def interval(data, P):
    # Write your code here
    # interval(data, 40) should output [-0.16604812797447732, 0.3160481279744774]
    expected = exp_return(data, P)
    vol = volatility(data, P)
    SE = vol / (3 ** (1/2))
    upper_bound = expected + 1.96 * SE
    lower_bound = expected - 1.96 * SE
    return [lower_bound, upper_bound]

Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [6]:
def exp_return_capm(beta, market_return, rf):
    # Write your code here
    # exp_return_capm(-0.6,0.1, 0.02) should output -0.028
    return (rf + beta * (market_return - rf))

### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [44]:
def clean_data(path):
    # Read the CSV file into a DataFrame
    data = pd.read_csv(path)
    
    data["RET"] = pd.to_numeric(data["RET"], errors='coerce')
    data = data.dropna()
    
    counts = data["PERMNO"].value_counts()

    permnos_to_keep = counts[counts >= 500].index.tolist()
    
    filtered_data = data[data["PERMNO"].isin(permnos_to_keep)]
    
    return filtered_data


Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [48]:
def avg_return(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    grouped = data.groupby("CUSIP")
    avg_returns = grouped["RET"].mean()
    
    output = pd.DataFrame(data = {
        "CUSIP" : avg_returns.index,
        "Average Return" : avg_returns.values
    })
    
    return output

In [50]:
def security_volatilities(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    grouped = data.groupby("CUSIP")
    vol = grouped["RET"].std()
    
    output = pd.DataFrame(data = {
        "CUSIP" : vol.index,
        "Volatility" : vol.values
    })
    
    
    return output

Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [56]:
def correlation(ret_data, vol_data):
    # Write your code here
    # correlation(rets, vols) should yield 0.44837090727059614
    final_data = pd.merge(ret_data, vol_data, on="CUSIP")
    avg_value = final_data["Average Return"]
    vol_value = final_data["Volatility"]
    value = avg_value.corr(vol_value)
    return value